In [1]:
!pip install huggingface_hub
!pip install datasets
!pip install transformers
!pip install sacrebleu
!pip insall accelerate
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.4 MB/s eta 0:00:00
ERROR: unknown command "insall" - maybe you meant "install"
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.6 MB/s eta 0:00:00


# **Necessary Imports**

In [2]:
import torch
from torch.utils.data import random_split
import datasets
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import gc
import torch
from transformers import AutoTokenizer, GenerationConfig
from transformers import (
    MBartForConditionalGeneration, MBartTokenizer, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer
)
from datasets import load_dataset
import evaluate
from transformers import MBartForConditionalGeneration

# **Dataset Loading and Preprocessing**

In [3]:
ds = load_dataset("SKNahin/bengali-transliteration-data")

split_datasets = ds['train'].train_test_split(test_size=0.2)

train_dataset = split_datasets['train']
valid_dataset = split_datasets['test']

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(valid_dataset)}")

max_input_length = max([len(ex['bn']) for ex in train_dataset])
max_output_length = max([len(ex['rm']) for ex in train_dataset])

print(f"Maximum input length (Bengali): {max_input_length}")
print(f"Maximum output length (Romanized Bengali): {max_output_length}")

train_dataset[:5]

README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/333k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5006 [00:00<?, ? examples/s]

Training set size: 4004
Validation set size: 1002
Maximum input length (Bengali): 246
Maximum output length (Romanized Bengali): 253


{'bn': ['লিঙ্ক কাজ করে না ',
  'রুট করার কোনো ওয়ে আছে ? ',
  'হুম গুড পোস্ট ব্রো',
  'ভাই,,,,,,ইউটিউব ভিপিএন মডেড করার একটা ভিডিওস দেন,,,,,,',
  'নাম্বার কই মিয়া '],
 'rm': ['link kaj kore na',
  'root korar kono way ase ?',
  'Hmmm gd post bro',
  'Vai,,,,,,youtube vpn moded korar ekta videos den,,,,,,',
  'Number koi mia']}

# **Initializing the Model**

In [4]:
checkpoint = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
sentences = ["আগে থেকেই জানি। থ্যাংকস ফর শেয়ার।" , "আমার ফোন স্যামসাং"]
tokenized_output = tokenizer(sentences)


print(tokenized_output)


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'input_ids': [[250004, 31636, 197562, 218577, 125, 6, 19293, 14329, 130180, 3458, 22741, 999, 61414, 125, 2], [250004, 29388, 59354, 12945, 14329, 4198, 16984, 17031, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [5]:
model = MBartForConditionalGeneration.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [6]:
max_input_length = 256
max_target_length = 256
source_lang = "rm"  
target_lang = "bn"  
tokenizer.src_lang = source_lang
tokenizer.tgt_lang = target_lang

def preprocess_function(examples):
    inputs = examples[source_lang]  
    targets = examples[target_lang]  


    model_inputs = tokenizer(
        inputs, max_length=max_input_length, truncation=True, padding="max_length"
    )
    labels = tokenizer(
        targets, max_length=max_target_length, truncation=True, padding="max_length"
    )


    sanitized_input_ids = [[id for id in seq if id >= 0] for seq in model_inputs["input_ids"]]
    sanitized_labels = [[id for id in seq if id >= 0] for seq in labels["input_ids"]]

    model_inputs["input_ids"] = sanitized_input_ids
    model_inputs["labels"] = sanitized_labels

    return model_inputs



In [7]:
tokenized_train_set = train_dataset.map(preprocess_function, batched=True)
tokenized_valid_set = valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/4004 [00:00<?, ? examples/s]

Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

In [8]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

metric = evaluate.load("sacrebleu")

print("Sample tokenized example:")
print(tokenized_train_set[0])

Sample tokenized example:
{'bn': 'লিঙ্ক কাজ করে না ', 'rm': 'link kaj kore na', 'input_ids': [3, 3126, 719, 20867, 24, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# **Train the Model**

In [9]:
!pip install --upgrade transformers accelerate

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 36.4 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 76.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [10]:
import torch

if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"GPU Available: {device_name}")
else:
    print("No GPU available.")

GPU Available: Tesla P100-PCIE-16GB


In [11]:
from accelerate import Accelerator


accelerator = Accelerator()

model, tokenizer, data_collator = accelerator.prepare(model, tokenizer, data_collator)
model.config.max_length = 200
model.config.early_stopping = True
model.config.num_beams = 5
model.config.forced_eos_token_id = 2

In [12]:
import torch
from tqdm.auto import tqdm
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"



training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/results",
    run_name="BanglishBartSiuu",
    eval_strategy="epoch",
    logging_dir="/kaggle/working/logs",
    logging_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=2,  
    per_device_eval_batch_size=2,  
    weight_decay=0.01,
    num_train_epochs=1, 
    predict_with_generate=True,
    save_steps=500,
    save_total_limit=3,
    fp16=True,
    report_to="none",
    disable_tqdm=False,
    gradient_checkpointing=True,  
)


training_args.gradient_accumulation_steps = 2


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_set,
    eval_dataset=tokenized_valid_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


# Manually clear CUDA cache
torch.cuda.empty_cache()

# Optionally, clear cache during training
trainer.train()

# Clear cache after training
torch.cuda.empty_cache()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  ):
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...

RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 3753363712 vs 3753363600